# DS Lab - Final Assignment: Robust Journey Planning #

## Load the data

In [1]:
#Importations.
import pandas as pd
import glob
from geopy import distance
import numpy as np

### Load the data published by the Open Data Platform Swiss Public Transport (https://opentransportdata.swiss).



In [2]:
#path on my computer of the files.
path = "2017-09/*.csv"

In [3]:
#useful list
data = []
#read each csv we had.
for fname in sorted(glob.glob(path)):
    d = pd.read_csv(fname, sep=';',low_memory=False)
    data.append(d)
    
#process the data from the csv files to a dataframe.
data = pd.concat(data)
data = pd.DataFrame(data)
data = data.reset_index()
data = data.drop(['index'], axis = 1)

In [53]:
#check the tail.
data.tail()

,BETRIEBSTAG,FAHRT_BEZEICHNER,BETREIBER_ID,BETREIBER_ABK,BETREIBER_NAME,PRODUKT_ID,LINIEN_ID,LINIEN_TEXT,UMLAUF_ID,VERKEHRSMITTEL_TEXT,...,FAELLT_AUS_TF,BPUIC,HALTESTELLEN_NAME,ANKUNFTSZEIT,AN_PROGNOSE,AN_PROGNOSE_STATUS,ABFAHRTSZEIT,AB_PROGNOSE,AB_PROGNOSE_STATUS,DURCHFAHRT_TF
14967129,30.09.2017,85:97:9:000,85:97,TRAVYS-ysc,Transports Vallée de Joux-Yverdon-Ste-Croix (Y...,Zug,9,R,NaN,R,...,False,8504294,Vuiteboeuf,30.09.2017 05:57,NaN,UNBEKANNT,30.09.2017 05:57,NaN,UNBEKANNT,False
14967130,30.09.2017,85:97:9:000,85:97,TRAVYS-ysc,Transports Vallée de Joux-Yverdon-Ste-Croix (Y...,Zug,9,R,NaN,R,...,False,8504295,Baulmes,30.09.2017 06:01,NaN,UNBEKANNT,30.09.2017 06:01,NaN,UNBEKANNT,False
14967131,30.09.2017,85:97:9:000,85:97,TRAVYS-ysc,Transports Vallée de Joux-Yverdon-Ste-Croix (Y...,Zug,9,R,NaN,R,...,False,8504296,Six-Fontaines,30.09.2017 06:05,NaN,UNBEKANNT,30.09.2017 06:05,NaN,UNBEKANNT,False
14967132,30.09.2017,85:97:9:000,85:97,TRAVYS-ysc,Transports Vallée de Joux-Yverdon-Ste-Croix (Y...,Zug,9,R,NaN,R,...,False,8504297,Trois-Villes,30.09.2017 06:12,NaN,UNBEKANNT,30.09.2017 06:12,NaN,UNBEKANNT,False
14967133,30.09.2017,85:97:9:000,85:97,TRAVYS-ysc,Transports Vallée de Joux-Yverdon-Ste-Croix (Y...,Zug,9,R,NaN,R,...,False,8504298,Ste-Croix,30.09.2017 06:21,NaN,UNBEKANNT,NaN,NaN,PROGNOSE,False


### Load the station list data.

In [38]:
#path.
station_path = "metadata/BFKOORD_GEO"

In [39]:
station_file  = open(station_path, "r")

In [40]:
BPUIC = []
long = []
lat = []
name = []

for line in station_file:
    l = line.split()
    BPUIC.append(l[0])
    long.append(l[1])
    lat.append(l[2])
    name.append(l[5])


In [43]:
d = {'BPUIC':BPUIC,
     'longitude' : long,
     'latitude' : lat,
     'station name': name}
station_df = pd.DataFrame(d)
station_df = station_df[['BPUIC','longitude','latitude','station name']]
station_df['BPUIC'] = station_df['BPUIC'].astype(np.int64)

In [44]:
station_df.head()

,BPUIC,longitude,latitude,station name
0,0000002,26.074412,44.446770,Bucuresti
1,0000003,1.811446,50.901549,Calais
2,0000004,1.075329,51.284212,Canterbury
3,0000005,-3.543547,50.729172,Exeter
4,0000007,9.733756,46.922368,"Fideris,"


In [48]:
zurich_coord = (47.407665036,8.540331172)
salamanca = (40.96, -5.50)

In [49]:
print(distance.distance(zurich_coord, salamanca).km)

1328.6915399458765


In [82]:
df = pd.merge(data,station_df,on='BPUIC',how='left')

In [83]:
df.tail()

,BETRIEBSTAG,FAHRT_BEZEICHNER,BETREIBER_ID,BETREIBER_ABK,BETREIBER_NAME,PRODUKT_ID,LINIEN_ID,LINIEN_TEXT,UMLAUF_ID,VERKEHRSMITTEL_TEXT,...,ANKUNFTSZEIT,AN_PROGNOSE,AN_PROGNOSE_STATUS,ABFAHRTSZEIT,AB_PROGNOSE,AB_PROGNOSE_STATUS,DURCHFAHRT_TF,longitude,latitude,station name
14967129,30.09.2017,85:97:9:000,85:97,TRAVYS-ysc,Transports Vallée de Joux-Yverdon-Ste-Croix (Y...,Zug,9,R,NaN,R,...,30.09.2017 05:57,NaN,UNBEKANNT,30.09.2017 05:57,NaN,UNBEKANNT,False,6.553607,46.803829,Vuiteboeuf
14967130,30.09.2017,85:97:9:000,85:97,TRAVYS-ysc,Transports Vallée de Joux-Yverdon-Ste-Croix (Y...,Zug,9,R,NaN,R,...,30.09.2017 06:01,NaN,UNBEKANNT,30.09.2017 06:01,NaN,UNBEKANNT,False,6.524330,46.788917,Baulmes
14967131,30.09.2017,85:97:9:000,85:97,TRAVYS-ysc,Transports Vallée de Joux-Yverdon-Ste-Croix (Y...,Zug,9,R,NaN,R,...,30.09.2017 06:05,NaN,UNBEKANNT,30.09.2017 06:05,NaN,UNBEKANNT,False,6.498853,46.774064,Six-Fontaines
14967132,30.09.2017,85:97:9:000,85:97,TRAVYS-ysc,Transports Vallée de Joux-Yverdon-Ste-Croix (Y...,Zug,9,R,NaN,R,...,30.09.2017 06:12,NaN,UNBEKANNT,30.09.2017 06:12,NaN,UNBEKANNT,False,6.531971,46.800268,Trois-Villes
14967133,30.09.2017,85:97:9:000,85:97,TRAVYS-ysc,Transports Vallée de Joux-Yverdon-Ste-Croix (Y...,Zug,9,R,NaN,R,...,30.09.2017 06:21,NaN,UNBEKANNT,NaN,NaN,PROGNOSE,False,6.500310,46.819668,Ste-Croix
